# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [2]:
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
Y = df[[c for c in df if c not in ['id', 'message', 'original', 'genre']]]

In [5]:
X.sample().values

array([ 'Soaking bednets with insecticides has been shown extremely effective in protecting people from malaria and can be distributed to the most rural and most vulnerable populations in rural villages where most deaths occur.'], dtype=object)

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [23]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

In [24]:
%%time

pipeline.fit(X_train, y_train)

CPU times: user 1min 2s, sys: 52 ms, total: 1min 2s
Wall time: 1min 2s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test)

In [26]:
for i, topic in enumerate(y_test):
    print('==============')
    print(f'Topic: {topic}')
    print(classification_report(y_test.iloc[:, i], y_pred[:, i], target_names=['1']))
    print('\n')

Topic: related
             precision    recall  f1-score   support

          1       0.65      0.34      0.45      1874

avg / total       0.77      0.79      0.77      7865



Topic: request
             precision    recall  f1-score   support

          1       0.88      0.98      0.93      6514

avg / total       0.87      0.88      0.86      7865



Topic: offer
             precision    recall  f1-score   support

          1       1.00      1.00      1.00      7838

avg / total       0.99      1.00      0.99      7865



Topic: aid_related
             precision    recall  f1-score   support

          1       0.73      0.88      0.80      4584

avg / total       0.74      0.74      0.73      7865



Topic: medical_help
             precision    recall  f1-score   support

          1       0.92      0.99      0.96      7216

avg / total       0.89      0.92      0.89      7865



Topic: medical_products
             precision    recall  f1-score   support

          1       0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 1
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
%%time 

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 1.0),
    'clf__estimator__max_depth': [None, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, y_train)

CPU times: user 19min 16s, sys: 716 ms, total: 19min 17s
Wall time: 19min 19s


In [26]:
pd.DataFrame(cv.cv_results_)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__max_depth,param_vect__max_df,param_vect__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,34.956132,0.377610,4.327914,0.048260,None,0.5,"(1, 1)","{'clf__estimator__max_depth': None, 'vect__max...",0.215792,0.230178,0.219879,0.221950,0.006053,3,0.816904,0.812490,0.811918,0.813770,0.002228
1,66.184216,1.235698,5.404735,0.034356,None,0.5,"(1, 2)","{'clf__estimator__max_depth': None, 'vect__max...",0.215138,0.229034,0.227236,0.223803,0.006170,1,0.795406,0.796142,0.796387,0.795978,0.000417
2,34.956738,0.156280,4.299161,0.038682,None,1,"(1, 1)","{'clf__estimator__max_depth': None, 'vect__max...",0.206637,0.210724,0.216282,0.211215,0.003953,4,0.806032,0.814533,0.806114,0.808893,0.003988
3,67.791129,2.181698,5.367083,0.029245,None,1,"(1, 2)","{'clf__estimator__max_depth': None, 'vect__max...",0.217590,0.224129,0.229034,0.223585,0.004688,2,0.799657,0.787723,0.795243,0.794207,0.004927
4,7.529365,0.051504,3.270318,0.039163,3,0.5,"(1, 1)","{'clf__estimator__max_depth': 3, 'vect__max_df...",0.191270,0.192251,0.192905,0.192142,0.000672,8,0.192823,0.190943,0.192987,0.192251,0.000927
5,9.248448,0.073546,3.750955,0.028715,3,0.5,"(1, 2)","{'clf__estimator__max_depth': 3, 'vect__max_df...",0.191924,0.193886,0.192905,0.192905,0.000801,5,0.193477,0.192415,0.192987,0.192960,0.000434
6,7.495645,0.020022,3.260122,0.045735,3,1,"(1, 1)","{'clf__estimator__max_depth': 3, 'vect__max_df...",0.190616,0.193886,0.192415,0.192306,0.001337,7,0.192169,0.192333,0.192742,0.192415,0.000241
7,9.179295,0.053609,3.737390,0.054411,3,1,"(1, 2)","{'clf__estimator__max_depth': 3, 'vect__max_df...",0.191270,0.193395,0.192415,0.192360,0.000868,6,0.193150,0.192169,0.192905,0.192742,0.000417


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
y_pred = cv.predict(X_test)

for i, topic in enumerate(y_test):
    print('==============')
    print(f'Topic: {topic}')
    accuracy = accuracy_score(y_pred=y_pred[:, i], y_true=y_test.iloc[:, i])
    precicsion = precision_score(y_pred=y_pred[:, i], y_true=y_test.iloc[:, i], average='macro')
    recall = recall_score(y_pred=y_pred[:, i], y_true=y_test.iloc[:, i], average='macro')
    print(f'accuracy: {accuracy}')
    print(f'precicsion: {precicsion}')
    print(f'recall: {recall}')
    print('\n')
    




Topic: related
accuracy: 0.9239669421487603
precicsion: 0.6153269221291736
recall: 0.5766980279550163


Topic: request
accuracy: 0.9553719008264463
precicsion: 0.9601918990969023
recall: 0.881846153846154


Topic: offer
accuracy: 0.9968213604577241
precicsion: 0.9984072375127421
recall: 0.7023809523809523


Topic: aid_related
accuracy: 0.9029879211697394
precicsion: 0.911870844983864
recall: 0.8901920721071601


Topic: medical_help
accuracy: 0.968340750158932
precicsion: 0.9725672571693333
recall: 0.793115008475804


Topic: medical_products
accuracy: 0.9800381436745073
precicsion: 0.9875154824738992
recall: 0.7913771035045977


Topic: search_and_rescue
accuracy: 0.989574062301335
precicsion: 0.9876791493296349
recall: 0.8163829912311957


Topic: security
accuracy: 0.99262555626192
precicsion: 0.990313606004589
recall: 0.7948993122354304


Topic: military
accuracy: 0.9871582962492054
precicsion: 0.9901202771857642
recall: 0.7966823420695159


Topic: child_alone
accuracy: 1.0
precicsion:

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, file='clf.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.